In [2]:
import pandas as pd
import numpy as np


#### Reading data

In [3]:
CA_data = pd.read_csv('CA_data.csv')

In [4]:
CA_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 81 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   File ID                                         379 non-null    object 
 1   Vehicle 1 Manufacturer                          379 non-null    object 
 2   Vehicle 1 Business Name                         379 non-null    object 
 3   Date                                            379 non-null    object 
 4   Time                                            379 non-null    object 
 5   Vehicle 1 Year of Manufacturing                 379 non-null    int64  
 6   Vehicle 1 Make                                  379 non-null    object 
 7   Vehicle 1 Model                                 379 non-null    object 
 8   Location of Accident                            379 non-null    object 
 9   City of Accident                           